# Tiny ImageNet - Augmentation using Imgaug, CLR, Custom Resnet

## Download data

In [ ]:
!pip install imgaug

    100% |████████████████████████████████| 17.3MB 2.7MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


In [ ]:
!pip install --upgrade scikit-image

    100% |████████████████████████████████| 26.3MB 1.8MB/s 
    100% |████████████████████████████████| 2.0MB 16.7MB/s 
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 4.1.1
    Uninstalling Pillow-4.1.1:
      Successfully uninstalled Pillow-4.1.1
  Found existing installation: scikit-image 0.13.1
    Uninstalling scikit-image-0.13.1:
      Successfully uninstalled scikit-image-0.13.1


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!cp '/content/gdrive/My Drive/App/Tiny/MyUtils.zip' /content
!unzip -q /content/MyUtils.zip

!cp '/content/gdrive/My Drive/App/Tiny/Mod/tiny-imagenet-200.zip' /content
!unzip -q /content/tiny-imagenet-200.zip

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Configurations

### Custom ResNet

In [ ]:
# import the necessary packages
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D

from keras.layers import GlobalAveragePooling2D
from keras.layers import SeparableConv2D

from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

'''
    Standard ResNet50 is too deep for Tiny ImageNet dataset.
    Let's build a custom one.

    1) Instead of 7x7 filter use 5,5 with stride 1
    2) Experiment about removing MaxPool in the entry block
       Reason: Don't want to reduce the input dimension.
    3) About repetition resuidal block - remove last set.
       Reason: it's too deep for this dataset.
       Also, we want to go beyond the object and identify scenes as well.
    4) About kernel count - experiment the following
        * [64, 128, 256]
        * or [128, 256, 512]
        * Make sure the params are well within the limit
    4) Remove Dense Layer
        * Use GAP
        * or 1x1 Conv.
        * By default, network uses 'Avg pool' before flatten.
          Do we need that if we go with GAP?
    5) Stretch goal: Experiment with SeparableConv2D() instead of Conv2D().
'''
class CustomResNet:
    @staticmethod
    def entry_block(input, num_filters):
        '''
        1. In the entry block, use filter 5x5 with stride 1 instead of 7x7.
        2. Optionally think about ignoring the Maxpool in the entry block.
        Idea is, input 64x64 => before the resuidal block, retain same input
        shape or reduced to just 32x32.
        '''
        ## CONV -> BN -> ACT -> MaxPool
        x = Conv2D(num_filters, (5,5), padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = ZeroPadding2D((1,1))(x) # next one is max pool, do prep the data accordingly.
        block_output = MaxPooling2D((3,3), strides=(2,2))(x)
        return block_output

    '''
    1x1 CONV -> 3x3 CONV -> 1x1 CONV
    '''
    @staticmethod
    def residual_block(input_data, num_filters, stride):

        short_cut = input_data

        # BN -> Act -> Conv2D(1,1)
        bn1 = BatchNormalization()(input_data)
        act1 = Activation('relu')(bn1)
        conv1 = Conv2D(int(num_filters * 0.25), (1,1))(act1)

        # BN -> Act -> Conv2D(3,3)
        bn2 = BatchNormalization()(conv1)
        act2 = Activation('relu')(bn2)
        conv2 = Conv2D(int(num_filters * 0.25), (3,3), strides=stride, padding='same')(act2)

        # BN -> Act -> Conv2D(1,1)
        bn3 = BatchNormalization()(conv2)
        act3 = Activation('relu')(bn3)
        conv3 = Conv2D(num_filters, (1,1))(act3)

        # need to add with short cut.
        x = add([conv3, short_cut])
        return x

    '''
    Entry block of the residual network, which needs to reduce
    the spatial volume. Basically, need to create the CONV layer
    in the short_cut-branch to match the dimension from the main-branch.
    '''
    @staticmethod
    def residual_block_reduce(input_data, num_filters, stride):
        short_cut = input_data

        # BN -> Act -> Conv2D(1,1)
        bn1 = BatchNormalization()(input_data)
        act1 = Activation('relu')(bn1)
        conv1 = Conv2D(int(num_filters * 0.25), (1, 1))(act1)

        # BN -> Act -> Conv2D(3,3)
        bn2 = BatchNormalization()(conv1)
        act2 = Activation('relu')(bn2)
        conv2 = Conv2D(int(num_filters * 0.25), (3, 3),
                       strides=stride, padding='same')(act2)

        # BN -> Act -> Conv2D(1,1)
        bn3 = BatchNormalization()(conv2)
        act3 = Activation('relu')(bn3)
        conv3 = Conv2D(num_filters, (1, 1))(act3)

        ## to match the dimension from the main-branch with short-cut-branch
        ## apply Conv 1,1 in the short-cut branch, so addition of layers is possible.
        short_cut = Conv2D(num_filters, (1,1), strides=stride)(act1)

        # need to add with short cut.
        x = add([conv3, short_cut])
        return x

    @staticmethod
    def build_resnet(width, height, depth, num_classes, stages, filters):
        """Builds a custom ResNet like architecture.
            Returns:
            The keras `Model`.
        """
        # get the input shape, channels last.
        inputShape = (width, height, depth)

        # get the input layer
        inputs = Input(shape=inputShape)

        ## before apply CONV, it's better to normalize the input
        ## use BatchNorm, by default, channel last
        x = BatchNormalization()(inputs)

        ## build entry block
        x = CustomResNet.entry_block(x, filters[0])

        ## build the residual blocks for different stages
        for i in range(0, len(stages)):
            stride = (1,1) if i == 0 else (2,2)

            # first block need to reduce the spatial size of input vol.
            # in the short-cut-branch to enable concatenation.
            x = CustomResNet.residual_block_reduce(x, filters[i+1], stride)

            # depth of the residual block.
            for j in range(0, stages[i]-1):
               x = CustomResNet.residual_block(x, filters[i+1], (1,1))
        
        ## BN -> Act -> Ave. pooling
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        block_output = x
        block_shape  = K.int_shape(x)

        #try GlobalAvgPool instead of AvgPool
        # input tensor 8x8x512
        x = SeparableConv2D(block_shape[3] // 2, (3, 3))(block_output)

        # reduce the volume to num_classes
        x = SeparableConv2D(num_classes, (3,3))(x)
        x = GlobalAveragePooling2D()(x)
         
        #x = AveragePooling2D((8,8))(x)
        

        ## outputs
        #x = Flatten()(x)
        #x = Dense(num_classes)(x)
        
        outputs = Activation('softmax')(x)
        

        ## build the model
        model = Model(inputs, outputs, name='custom_resnet')
        return model



### Imports and Configurations

In [ ]:
import keras
import os
import numpy as np
import pandas as pd

nn

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

## Learning Rate Scheduler

from keras.callbacks import LearningRateScheduler

##from keras_preprocessing.image import ImageDataGenerator

from MyUtils.Dataset import DatasetLoader
from MyUtils.CustomResNet import CustomResNet
from MyUtils.CLR import CyclicLR


from keras.optimizers import SGD
import cv2

## configurations ##
ROOT_DIR='/content'
IMAGE_ROOT_DIR=os.path.join(ROOT_DIR, 'tiny-imagenet-200')

# utilities.
UTILS_ROOT_DIR='/content'
MY_UTILS_DIR= os.path.join(UTILS_ROOT_DIR,'MyUtils')
SUPPORT_FILES_DIR=os.path.join(MY_UTILS_DIR, 'SupportFiles')


train_data_dir = os.path.join(IMAGE_ROOT_DIR,'train')
validation_data_dir = os.path.join(IMAGE_ROOT_DIR, 'val')
num_train_samples = 100000
num_validation_samples = 10000

## colab crashes, so need to multiple runs - may be 25 epochs three times.
epochs = 50
batch_size = 256

img_height = 64
img_width  = 64
num_channel = 3
num_classes = 200

## in the case of poly decay.
INIT_LR = 1e-1

Using TensorFlow backend.


## Build Model

In [ ]:
## 1. Use CustomResNet - which uses GAP, SeparableConv2D.
## 2. Also, as we need shorter resnet with shortcut connections, 
##    remove the last repetition of residual block.
## 3. As we need wider network - use 128,256,512 depth'channels in residual blocks.
##    Need to experiment [256,512,1024] as well to see whether that provides better accuracy.
###

model = CustomResNet.build_resnet(64, 64,3, 200, (3,4,6), (64,128,256,512))
                     
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   4864        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 64, 64, 64)   256         conv2d_1[0][0]                   
_____________________________________

In [ ]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=INIT_LR, momentum=0.9),
              metrics=['accuracy'])

## Cyclic LR

In [ ]:
## define the poly decay
## input is epoch number and returns new alpha (lr)
def poly_decay(epoch):
  maxEpochs = epochs
  base_lr   = INIT_LR
  power     = 1.0
  
  # compute new lr
  new_lr = base_lr * ( 1 - (epoch / float(maxEpochs))) ** power
  
  # new learning rate
  return new_lr
  
##### CLR #########
## Experiment with CLR instead of poly decay
clr = CyclicLR(base_lr=0.001, max_lr=0.1, mode='exp_range')

In [ ]:
## to save the best model 

checkpoint = ModelCheckpoint('/content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_custom_best_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True)

##callback_list = [checkpoint, LearningRateScheduler(poly_decay)]

callback_list = [checkpoint, clr]

## Genertors for Training and Validation

In [ ]:
'''
rMean = 122.46
gMean = 114.25
bMean = 101.36

def mean_subtraction(img):
    (r_init, g_init, b_init) = cv2.split(img.astype('float32'))
    
    r = r_init - rMean
    g = g_init - gMean
    b = b_init - bMean
    
    new_img = cv2.merge([r, g, b])
    return new_img
  

## preprocess image and training augmentation
train_datagen = ImageDataGenerator(rotation_range=18, 
                                   zoom_range=0.15, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                   shear_range=0.15, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest',
                                    preprocessing_function=mean_subtraction)

'''
## Use imgaug augmentations.

img_aug = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0.0, 3.0)),    # blur the images
    iaa.Fliplr(0.5),                       # horizontally flip 50% of the images
    iaa.Crop(px=(0, 16)),                  # crop images from each side by 0 to 16px (randomly chosen)
    iaa.ContrastNormalization((0.5, 1.5)), # normalize the contrast
    iaa.CropAndPad(percent=(-0.25, 0.25)), # crop and pad with black 
    iaa.Affine(scale=(0.5, 1.5)),          # zoom in and zoom out 50 to 150%
    iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)}), # translate x & y 20 pixels independently
    iaa.Affine(rotate=(-18, 18)),          # rotate
    iaa.Affine(shear=(-15, 15)),           # shear
    iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)) # sharpen the image
])

train_datagen = ImageDataGenerator(preprocessing_function=img_aug.augment_image)

## train generator
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(img_width, img_height),
                    batch_size=batch_size,
                    class_mode='categorical')

label_dict = train_generator.class_indices

print(label_dict.keys())
print(label_dict.values())

## validation generator
#validation_datagen = ImageDataGenerator(preprocessing_function=mean_subtraction)
validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 100000 images belonging to 200 classes.
dict_keys(['n01443537', 'n01629819', 'n01641577', 'n01644900', 'n01698640', 'n01742172', 'n01768244', 'n01770393', 'n01774384', 'n01774750', 'n01784675', 'n01855672', 'n01882714', 'n01910747', 'n01917289', 'n01944390', 'n01945685', 'n01950731', 'n01983481', 'n01984695', 'n02002724', 'n02056570', 'n02058221', 'n02074367', 'n02085620', 'n02094433', 'n02099601', 'n02099712', 'n02106662', 'n02113799', 'n02123045', 'n02123394', 'n02124075', 'n02125311', 'n02129165', 'n02132136', 'n02165456', 'n02190166', 'n02206856', 'n02226429', 'n02231487', 'n02233338', 'n02236044', 'n02268443', 'n02279972', 'n02281406', 'n02321529', 'n02364673', 'n02395406', 'n02403003', 'n02410509', 'n02415577', 'n02423022', 'n02437312', 'n02480495', 'n02481823', 'n02486410', 'n02504458', 'n02509815', 'n02666196', 'n02669723', 'n02699494', 'n02730930', 'n02769748', 'n02788148', 'n02791270', 'n02793495', 'n02795169', 'n02802426', 'n02808440', 'n02814533', 'n02814860', 'n02815

## Training

### Epochs 1- 50

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size,
    callbacks = callback_list
)

Epoch 1/50
390/390 [==============================] - 587s 2s/step - loss: 5.2986 - acc: 0.0045 - val_loss: 5.2982 - val_acc: 0.0057

Epoch 00001: val_acc improved from -inf to 0.00575, saving model to /content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_custom_best_model.hdf5
Epoch 2/50
390/390 [==============================] - 573s 1s/step - loss: 5.2810 - acc: 0.0076 - val_loss: 5.2523 - val_acc: 0.0130

Epoch 00002: val_acc improved from 0.00575 to 0.01302, saving model to /content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_custom_best_model.hdf5
Epoch 3/50
390/390 [==============================] - 583s 1s/step - loss: 5.0399 - acc: 0.0217 - val_loss: 5.0191 - val_acc: 0.0300

Epoch 00003: val_acc improved from 0.01302 to 0.02997, saving model to /content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_custom_best_model.hdf5
Epoch 4/50
390/390 [==============================] - 583s 1s/step - loss: 4.8272 - acc: 0.0427 - val_loss: 5.4261 - val_acc: 0.0380

Epoch 00004: val_acc improved from 

In [ ]:
model.save('tiny_aug_e50.hdf5')

NameError: ignored

In [ ]:
!cp /content/tiny_aug_e50.hdf5 '/content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_aug_e50.hdf5'

### Run for another 50 epochs

In [ ]:
!cp '/content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_aug_e50.hdf5'  /content/tiny_aug_e50.hdf5

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('tiny_aug_e50.hdf5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [ ]:
## Use imgaug augmentations.

# instead of applying the transformations sequentially, pick one and augment.
#img_aug = iaa.Sequential([
    
img_aug = iaa.OneOf([
    iaa.GaussianBlur(sigma=(0.0, 3.0)),    # blur the images
    iaa.Fliplr(0.5),                       # horizontally flip 50% of the images
    iaa.Crop(px=(0, 16)),                  # crop images from each side by 0 to 16px (randomly chosen)
    iaa.ContrastNormalization((0.5, 1.5)), # normalize the contrast
    iaa.CropAndPad(percent=(-0.25, 0.25)), # crop and pad with black 
    iaa.Affine(scale=(0.5, 1.5)),          # zoom in and zoom out 50 to 150%
    iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)}), # translate x & y 20 pixels independently
    iaa.Affine(rotate=(-18, 18)),          # rotate
    iaa.Affine(shear=(-15, 15)),           # shear
    iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)) # sharpen the image
])

train_datagen = ImageDataGenerator(preprocessing_function=img_aug.augment_image)

## train generator
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(img_width, img_height),
                    batch_size=batch_size,
                    class_mode='categorical')

label_dict = train_generator.class_indices

print(label_dict.keys())
print(label_dict.values())

## validation generator
#validation_datagen = ImageDataGenerator(preprocessing_function=mean_subtraction)
validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 100000 images belonging to 200 classes.
dict_keys(['n01443537', 'n01629819', 'n01641577', 'n01644900', 'n01698640', 'n01742172', 'n01768244', 'n01770393', 'n01774384', 'n01774750', 'n01784675', 'n01855672', 'n01882714', 'n01910747', 'n01917289', 'n01944390', 'n01945685', 'n01950731', 'n01983481', 'n01984695', 'n02002724', 'n02056570', 'n02058221', 'n02074367', 'n02085620', 'n02094433', 'n02099601', 'n02099712', 'n02106662', 'n02113799', 'n02123045', 'n02123394', 'n02124075', 'n02125311', 'n02129165', 'n02132136', 'n02165456', 'n02190166', 'n02206856', 'n02226429', 'n02231487', 'n02233338', 'n02236044', 'n02268443', 'n02279972', 'n02281406', 'n02321529', 'n02364673', 'n02395406', 'n02403003', 'n02410509', 'n02415577', 'n02423022', 'n02437312', 'n02480495', 'n02481823', 'n02486410', 'n02504458', 'n02509815', 'n02666196', 'n02669723', 'n02699494', 'n02730930', 'n02769748', 'n02788148', 'n02791270', 'n02793495', 'n02795169', 'n02802426', 'n02808440', 'n02814533', 'n02814860', 'n02815

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size,
    callbacks = callback_list,
    initial_epoch=50
)

Epoch 51/100
390/390 [==============================] - 348s 893ms/step - loss: 2.5408 - acc: 0.4058 - val_loss: 2.4698 - val_acc: 0.4139

Epoch 00051: val_acc improved from -inf to 0.41390, saving model to /content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_custom_best_model.hdf5
Epoch 52/100
390/390 [==============================] - 333s 855ms/step - loss: 2.3231 - acc: 0.4436 - val_loss: 2.5589 - val_acc: 0.4005

Epoch 00052: val_acc did not improve from 0.41390
Epoch 53/100
390/390 [==============================] - 332s 851ms/step - loss: 2.2889 - acc: 0.4471 - val_loss: 2.5334 - val_acc: 0.4051

Epoch 00053: val_acc did not improve from 0.41390
Epoch 54/100
390/390 [==============================] - 332s 850ms/step - loss: 2.2776 - acc: 0.4477 - val_loss: 2.5372 - val_acc: 0.4047

Epoch 00054: val_acc did not improve from 0.41390
Epoch 55/100
390/390 [==============================] - 333s 853ms/step - loss: 2.2608 - acc: 0.4514 - val_loss: 2.8179 - val_acc: 0.3541

Epoch 00055: v

In [ ]:
model.save('imgaug_clr_e100.hdf5')

In [ ]:
!cp imgaug_clr_e100.hdf5 '/content/gdrive/My Drive/App/Tiny/SGD/imgaug'

### Instead of Augment OneOf, apply Sequential and run for 25 epochs

In [ ]:
## Use imgaug augmentations.

# instead of applying the transformations sequentially, pick one and augment.
#img_aug = iaa.Sequential([
    
#img_aug = iaa.OneOf([
img_aug = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0.0, 3.0)),    # blur the images
    iaa.Fliplr(0.5),                       # horizontally flip 50% of the images
    iaa.Crop(px=(0, 16)),                  # crop images from each side by 0 to 16px (randomly chosen)
    iaa.ContrastNormalization((0.5, 1.5)), # normalize the contrast
    iaa.CropAndPad(percent=(-0.25, 0.25)), # crop and pad with black 
    iaa.Affine(scale=(0.5, 1.5)),          # zoom in and zoom out 50 to 150%
    iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)}), # translate x & y 20 pixels independently
    iaa.Affine(rotate=(-18, 18)),          # rotate
    iaa.Affine(shear=(-15, 15)),           # shear
    iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)) # sharpen the image
])

train_datagen = ImageDataGenerator(preprocessing_function=img_aug.augment_image)

## train generator
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(img_width, img_height),
                    batch_size=batch_size,
                    class_mode='categorical')

label_dict = train_generator.class_indices

print(label_dict.keys())
print(label_dict.values())

## validation generator
#validation_datagen = ImageDataGenerator(preprocessing_function=mean_subtraction)
validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 100000 images belonging to 200 classes.
dict_keys(['n01443537', 'n01629819', 'n01641577', 'n01644900', 'n01698640', 'n01742172', 'n01768244', 'n01770393', 'n01774384', 'n01774750', 'n01784675', 'n01855672', 'n01882714', 'n01910747', 'n01917289', 'n01944390', 'n01945685', 'n01950731', 'n01983481', 'n01984695', 'n02002724', 'n02056570', 'n02058221', 'n02074367', 'n02085620', 'n02094433', 'n02099601', 'n02099712', 'n02106662', 'n02113799', 'n02123045', 'n02123394', 'n02124075', 'n02125311', 'n02129165', 'n02132136', 'n02165456', 'n02190166', 'n02206856', 'n02226429', 'n02231487', 'n02233338', 'n02236044', 'n02268443', 'n02279972', 'n02281406', 'n02321529', 'n02364673', 'n02395406', 'n02403003', 'n02410509', 'n02415577', 'n02423022', 'n02437312', 'n02480495', 'n02481823', 'n02486410', 'n02504458', 'n02509815', 'n02666196', 'n02669723', 'n02699494', 'n02730930', 'n02769748', 'n02788148', 'n02791270', 'n02793495', 'n02795169', 'n02802426', 'n02808440', 'n02814533', 'n02814860', 'n02815

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size,
    callbacks = callback_list
    #initial_epoch=51
)

Epoch 1/25
390/390 [==============================] - 594s 2s/step - loss: 3.6614 - acc: 0.2223 - val_loss: 2.6241 - val_acc: 0.4238

Epoch 00001: val_acc did not improve from 0.50800
Epoch 2/25
390/390 [==============================] - 595s 2s/step - loss: 3.3050 - acc: 0.2682 - val_loss: 2.5724 - val_acc: 0.4321

Epoch 00002: val_acc did not improve from 0.50800
Epoch 3/25
390/390 [==============================] - 576s 1s/step - loss: 3.2494 - acc: 0.2748 - val_loss: 2.6735 - val_acc: 0.4148

Epoch 00003: val_acc did not improve from 0.50800
Epoch 4/25
390/390 [==============================] - 588s 2s/step - loss: 3.2059 - acc: 0.2800 - val_loss: 2.8006 - val_acc: 0.3996

Epoch 00004: val_acc did not improve from 0.50800
Epoch 5/25
390/390 [==============================] - 580s 1s/step - loss: 3.2125 - acc: 0.2815 - val_loss: 2.8311 - val_acc: 0.3694

Epoch 00005: val_acc did not improve from 0.50800
Epoch 6/25
390/390 [==============================] - 593s 2s/step - loss: 3.217

KeyboardInterrupt: ignored

### Resume Training

In [ ]:
## Use imgaug augmentations.

# instead of applying the transformations sequentially, pick one and augment.
#img_aug = iaa.Sequential([
    
#img_aug = iaa.OneOf([
#img_aug = iaa.Sequential([
img_aug = iaa.SomeOf(2, [
    iaa.GaussianBlur(sigma=(0.0, 3.0)),    # blur the images
    iaa.Fliplr(0.5),                       # horizontally flip 50% of the images
    iaa.Crop(px=(0, 16)),                  # crop images from each side by 0 to 16px (randomly chosen)
    iaa.ContrastNormalization((0.5, 1.5)), # normalize the contrast
    iaa.CropAndPad(percent=(-0.25, 0.25)), # crop and pad with black 
    iaa.Affine(scale=(0.5, 1.5)),          # zoom in and zoom out 50 to 150%
    iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)}), # translate x & y 20 pixels independently
    iaa.Affine(rotate=(-18, 18)),          # rotate
    iaa.Affine(shear=(-15, 15)),           # shear
    iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)) # sharpen the image
])

train_datagen = ImageDataGenerator(preprocessing_function=img_aug.augment_image)

## train generator
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(img_width, img_height),
                    batch_size=batch_size,
                    class_mode='categorical')

label_dict = train_generator.class_indices

print(label_dict.keys())
print(label_dict.values())

## validation generator
#validation_datagen = ImageDataGenerator(preprocessing_function=mean_subtraction)
validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 100000 images belonging to 200 classes.
dict_keys(['n01443537', 'n01629819', 'n01641577', 'n01644900', 'n01698640', 'n01742172', 'n01768244', 'n01770393', 'n01774384', 'n01774750', 'n01784675', 'n01855672', 'n01882714', 'n01910747', 'n01917289', 'n01944390', 'n01945685', 'n01950731', 'n01983481', 'n01984695', 'n02002724', 'n02056570', 'n02058221', 'n02074367', 'n02085620', 'n02094433', 'n02099601', 'n02099712', 'n02106662', 'n02113799', 'n02123045', 'n02123394', 'n02124075', 'n02125311', 'n02129165', 'n02132136', 'n02165456', 'n02190166', 'n02206856', 'n02226429', 'n02231487', 'n02233338', 'n02236044', 'n02268443', 'n02279972', 'n02281406', 'n02321529', 'n02364673', 'n02395406', 'n02403003', 'n02410509', 'n02415577', 'n02423022', 'n02437312', 'n02480495', 'n02481823', 'n02486410', 'n02504458', 'n02509815', 'n02666196', 'n02669723', 'n02699494', 'n02730930', 'n02769748', 'n02788148', 'n02791270', 'n02793495', 'n02795169', 'n02802426', 'n02808440', 'n02814533', 'n02814860', 'n02815

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size,
    callbacks = callback_list,
    initial_epoch=11
)

Epoch 12/25
390/390 [==============================] - 342s 878ms/step - loss: 1.5802 - acc: 0.5958 - val_loss: 2.2512 - val_acc: 0.4951

Epoch 00012: val_acc did not improve from 0.50800
Epoch 13/25
390/390 [==============================] - 341s 875ms/step - loss: 1.3707 - acc: 0.6385 - val_loss: 2.3611 - val_acc: 0.4912

Epoch 00013: val_acc did not improve from 0.50800
Epoch 14/25
390/390 [==============================] - 337s 865ms/step - loss: 1.3082 - acc: 0.6551 - val_loss: 2.4120 - val_acc: 0.4813

Epoch 00014: val_acc did not improve from 0.50800
Epoch 15/25
390/390 [==============================] - 337s 864ms/step - loss: 1.3178 - acc: 0.6521 - val_loss: 2.6790 - val_acc: 0.4613

Epoch 00015: val_acc did not improve from 0.50800
Epoch 16/25
390/390 [==============================] - 338s 866ms/step - loss: 1.3948 - acc: 0.6326 - val_loss: 2.6978 - val_acc: 0.4417

Epoch 00016: val_acc did not improve from 0.50800
Epoch 17/25
390/390 [==============================] - 342s 

In [ ]:
model.save('imgaug_clr_e136.hdf5')

In [ ]:
!cp imgaug_clr_e136.hdf5 '/content/gdrive/My Drive/App/Tiny/SGD/imgaug'

### Try Class weights

In [ ]:
!cp '/content/gdrive/My Drive/App/Tiny/SGD/imgaug/imgaug_clr_e136.hdf5' /content

In [ ]:
!cp '/content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_custom_best_model.hdf5' /content

In [ ]:
from keras.models import load_model

model = load_model('/content/tiny_custom_best_model.hdf5')

In [ ]:
validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)


score=model.evaluate_generator(generator=validation_generator, steps = num_validation_samples)
print(score)

Found 10000 images belonging to 200 classes.
[2.5369316147575662, 0.5115]


#### Predict

In [ ]:
# get the predictions
pred = model.predict_generator(validation_generator, steps = num_validation_samples, verbose=1)

# get the predict class
pred_class_indices = np.argmax(pred,axis=1)

# get actual labels
actuals = validation_generator.labels

cmpare = np.equal(actuals, pred_class_indices)

10000/10000 [==============================] - 239s 24ms/step


#### Get Class Weights

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def get_class_weights(actuals, pred, class_indices, alpha=1):
  clf_rep = precision_recall_fscore_support(actuals, pred)
  
  # build the dictionary
  clf_dict = {
             "precision" :clf_rep[0].round(2),
             "recall" : clf_rep[1].round(2),
             "f1-score" : clf_rep[2].round(2),
             "support" : clf_rep[3]
            }
  clf_df = pd.DataFrame(clf_dict, index = class_indices)
  
  # build the class weights
  class_weights = dict()
  for class_idx in class_indices:
    
    f1_score = clf_df['f1-score'][class_idx]
    
    # weight = 1 + alpha * (1 - f1-score)
    class_weights[class_idx] = (1 + (alpha * (1 - f1_score)) ).round(2)
      
  #print(class_weights)
  return(class_weights)

In [ ]:
class_weights = get_class_weights(actuals, 
                                  pred_class_indices, 
                                  validation_generator.class_indices.values())

In [ ]:
class_weights.values()

dict_values([1.23, 1.29, 1.5, 1.65, 1.52, 1.71, 1.38, 1.52, 1.31, 1.6, 1.56, 1.44, 1.37, 1.36, 1.32, 1.53, 1.7, 1.38, 1.46, 1.43, 1.29, 1.23, 1.2, 1.23, 1.69, 1.33, 1.56, 1.73, 1.45, 1.72, 1.55, 1.41, 1.71, 1.6, 1.43, 1.45, 1.27, 1.5, 1.35, 1.52, 1.71, 1.53, 1.49, 1.47, 1.15, 1.22, 1.44, 1.39, 1.63, 1.55, 1.31, 1.5, 1.24, 1.39, 1.31, 1.45, 1.54, 1.37, 1.27, 1.55, 1.31, 1.53, 1.71, 1.58, 1.78, 1.67, 1.39, 1.69, 1.4, 1.64, 1.56, 1.33, 1.59, 1.55, 1.45, 1.65, 1.52, 1.76, 1.33, 1.65, 1.7, 1.22, 1.43, 1.53, 1.56, 1.6, 1.51, 1.56, 1.79, 1.61, 1.38, 1.41, 1.51, 1.49, 1.57, 1.54, 1.64, 1.44, 1.38, 1.7, 1.76, 1.33, 1.55, 1.3, 1.63, 1.68, 1.6, 1.36, 1.31, 1.45, 1.45, 1.38, 1.58, 1.52, 1.62, 1.27, 1.61, 1.45, 1.37, 1.67, 1.65, 1.49, 1.72, 1.58, 1.27, 1.66, 1.41, 1.55, 1.53, 1.5, 1.57, 1.83, 1.83, 1.35, 1.5, 1.78, 1.54, 1.69, 1.79, 1.77, 1.52, 1.6, 1.6, 1.35, 1.55, 1.21, 1.42, 1.58, 1.44, 1.44, 1.56, 1.65, 1.53, 1.43, 1.42, 1.44, 1.52, 1.49, 1.55, 1.86, 1.59, 1.45, 1.39, 1.47, 1.48, 1.28, 1.22, 1.

#### Resume Training using Class Weights

In [ ]:
## define the poly decay
## input is epoch number and returns new alpha (lr)
def poly_decay(epoch):
  maxEpochs = epochs
  base_lr   = INIT_LR
  power     = 1.0
  
  # compute new lr
  new_lr = base_lr * ( 1 - (epoch / float(maxEpochs))) ** power
  
  # new learning rate
  return new_lr
  
##### CLR #########
## Experiment with CLR instead of poly decay
clr = CyclicLR(base_lr=0.001, max_lr=0.1, mode='exp_range')

In [ ]:
## to save the best model 

checkpoint = ModelCheckpoint('/content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_class_weights_best_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True)

##callback_list = [checkpoint, LearningRateScheduler(poly_decay)]

callback_list = [checkpoint, clr]

In [ ]:


# instead of applying the transformations sequentially, pick one and augment.
#img_aug = iaa.Sequential([
    
#img_aug = iaa.OneOf([
#img_aug = iaa.Sequential([
img_aug = iaa.SomeOf(2, [
    iaa.GaussianBlur(sigma=(0.0, 3.0)),    # blur the images
    iaa.Fliplr(0.5),                       # horizontally flip 50% of the images
    iaa.Crop(px=(0, 16)),                  # crop images from each side by 0 to 16px (randomly chosen)
    iaa.ContrastNormalization((0.5, 1.5)), # normalize the contrast
    iaa.CropAndPad(percent=(-0.25, 0.25)), # crop and pad with black 
    iaa.Affine(scale=(0.5, 1.5)),          # zoom in and zoom out 50 to 150%
    iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)}), # translate x & y 20 pixels independently
    iaa.Affine(rotate=(-18, 18)),          # rotate
    iaa.Affine(shear=(-15, 15)),           # shear
    iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)) # sharpen the image
])

train_datagen = ImageDataGenerator(preprocessing_function=img_aug.augment_image)

## train generator
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(img_width, img_height),
                    batch_size=batch_size,
                    class_mode='categorical')

label_dict = train_generator.class_indices

print(label_dict.keys())
print(label_dict.values())

## validation generator
#validation_datagen = ImageDataGenerator(preprocessing_function=mean_subtraction)
validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 100000 images belonging to 200 classes.
dict_keys(['n01443537', 'n01629819', 'n01641577', 'n01644900', 'n01698640', 'n01742172', 'n01768244', 'n01770393', 'n01774384', 'n01774750', 'n01784675', 'n01855672', 'n01882714', 'n01910747', 'n01917289', 'n01944390', 'n01945685', 'n01950731', 'n01983481', 'n01984695', 'n02002724', 'n02056570', 'n02058221', 'n02074367', 'n02085620', 'n02094433', 'n02099601', 'n02099712', 'n02106662', 'n02113799', 'n02123045', 'n02123394', 'n02124075', 'n02125311', 'n02129165', 'n02132136', 'n02165456', 'n02190166', 'n02206856', 'n02226429', 'n02231487', 'n02233338', 'n02236044', 'n02268443', 'n02279972', 'n02281406', 'n02321529', 'n02364673', 'n02395406', 'n02403003', 'n02410509', 'n02415577', 'n02423022', 'n02437312', 'n02480495', 'n02481823', 'n02486410', 'n02504458', 'n02509815', 'n02666196', 'n02669723', 'n02699494', 'n02730930', 'n02769748', 'n02788148', 'n02791270', 'n02793495', 'n02795169', 'n02802426', 'n02808440', 'n02814533', 'n02814860', 'n02815

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size,
    callbacks = callback_list,
    initial_epoch=25,
    class_weight = class_weights
)

Epoch 26/50
390/390 [==============================] - 345s 886ms/step - loss: 1.8887 - acc: 0.6729 - val_loss: 2.8290 - val_acc: 0.4472

Epoch 00026: val_acc improved from -inf to 0.44725, saving model to /content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_class_weights_best_model.hdf5
Epoch 27/50
390/390 [==============================] - 340s 873ms/step - loss: 2.2284 - acc: 0.6176 - val_loss: 2.9630 - val_acc: 0.4186

Epoch 00027: val_acc did not improve from 0.44725
Epoch 28/50
390/390 [==============================] - 340s 872ms/step - loss: 2.4635 - acc: 0.5835 - val_loss: 3.0578 - val_acc: 0.3903

Epoch 00028: val_acc did not improve from 0.44725
Epoch 29/50
390/390 [==============================] - 340s 872ms/step - loss: 2.5190 - acc: 0.5745 - val_loss: 2.7509 - val_acc: 0.4370

Epoch 00029: val_acc did not improve from 0.44725
Epoch 30/50
390/390 [==============================] - 339s 870ms/step - loss: 2.2513 - acc: 0.6177 - val_loss: 2.7547 - val_acc: 0.4506

Epoch 00030:

#### Give more weightage to failing classes (Make alpha as 2 or 3).

In [ ]:
!cp '/content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_class_weights_best_model.hdf5' /content

In [ ]:
from keras.models import load_model

model = load_model('/content/tiny_class_weights_best_model.hdf5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [ ]:
validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

# get the predictions
pred = model.predict_generator(validation_generator, steps = num_validation_samples, verbose=1)

# get the predict class
pred_class_indices = np.argmax(pred,axis=1)

# get actual labels
actuals = validation_generator.labels

cmpare = np.equal(actuals, pred_class_indices)

sum(cmpare)

Found 10000 images belonging to 200 classes.
10000/10000 [==============================] - 236s 24ms/step


5122

#### Train for few more epochs

In [ ]:
class_weights = get_class_weights(actuals, 
                                  pred_class_indices, 
                                  validation_generator.class_indices.values(),
                                  alpha = 3) ## give more weightage to failing classes.

In [ ]:
class_weights.values()

dict_values([1.78, 1.78, 2.38, 2.92, 2.47, 3.25, 1.99, 2.59, 1.72, 2.56, 2.47, 2.32, 2.05, 1.72, 1.84, 2.56, 3.1, 2.23, 2.41, 2.11, 1.87, 1.57, 1.9, 1.87, 3.07, 1.96, 2.5, 3.13, 2.41, 2.95, 2.77, 2.29, 3.04, 2.38, 1.96, 2.32, 1.84, 2.29, 1.81, 2.56, 3.16, 2.47, 2.56, 2.41, 1.42, 1.6, 2.35, 2.08, 2.89, 2.77, 1.99, 2.29, 1.66, 2.02, 1.9, 1.99, 2.62, 2.14, 1.75, 2.74, 1.87, 2.5, 3.19, 2.62, 3.16, 3.16, 2.38, 2.86, 2.2, 2.95, 2.41, 2.05, 2.56, 2.65, 2.32, 2.98, 2.59, 3.07, 1.96, 3.13, 3.13, 1.66, 2.44, 2.83, 2.86, 2.71, 2.56, 2.83, 3.25, 2.71, 2.17, 2.02, 2.59, 2.53, 2.56, 2.71, 2.65, 2.5, 2.2, 2.98, 3.13, 1.99, 2.53, 1.84, 2.89, 2.98, 2.89, 1.87, 1.78, 2.11, 2.59, 2.14, 2.8, 2.62, 2.71, 1.66, 2.56, 2.35, 1.93, 2.89, 2.62, 2.29, 3.04, 2.98, 1.93, 3.25, 2.26, 2.53, 2.74, 2.35, 2.77, 3.73, 3.46, 1.87, 2.44, 3.4, 2.71, 2.8, 3.37, 3.25, 2.47, 2.8, 2.8, 1.69, 2.65, 1.6, 2.11, 2.77, 2.35, 2.29, 2.65, 2.74, 2.5, 2.11, 2.44, 2.38, 2.29, 2.47, 2.8, 3.49, 2.71, 2.59, 2.23, 2.32, 2.11, 1.81, 1.54, 2.

In [ ]:
# instead of applying the transformations sequentially, pick one and augment.
#img_aug = iaa.Sequential([
    
#img_aug = iaa.OneOf([
#img_aug = iaa.Sequential([
img_aug = iaa.SomeOf(2, [
    iaa.GaussianBlur(sigma=(0.0, 3.0)),    # blur the images
    iaa.Fliplr(0.5),                       # horizontally flip 50% of the images
    iaa.Crop(px=(0, 16)),                  # crop images from each side by 0 to 16px (randomly chosen)
    iaa.ContrastNormalization((0.5, 1.5)), # normalize the contrast
    iaa.CropAndPad(percent=(-0.25, 0.25)), # crop and pad with black 
    iaa.Affine(scale=(0.5, 1.5)),          # zoom in and zoom out 50 to 150%
    iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)}), # translate x & y 20 pixels independently
    iaa.Affine(rotate=(-18, 18)),          # rotate
    iaa.Affine(shear=(-15, 15)),           # shear
    iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)), # sharpen the image
    
    # Either drop randomly 1 to 10% of all pixels (i.e. set
    # them to black) or drop them on an image with 2-5% percent
    # of the original size, leading to large dropped
    # rectangles.
    iaa.OneOf([
               iaa.Dropout((0.01, 0.1), per_channel=0.5),
               iaa.CoarseDropout(
                        (0.03, 0.15), size_percent=(0.02, 0.05),
                        per_channel=0.2
                    ),
                ]),  
], random_order=True)

train_datagen = ImageDataGenerator(preprocessing_function=img_aug.augment_image)

## train generator
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(img_width, img_height),
                    batch_size=batch_size,
                    class_mode='categorical')

label_dict = train_generator.class_indices

print(label_dict.keys())
print(label_dict.values())

## validation generator
#validation_datagen = ImageDataGenerator(preprocessing_function=mean_subtraction)
validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 100000 images belonging to 200 classes.
dict_keys(['n01443537', 'n01629819', 'n01641577', 'n01644900', 'n01698640', 'n01742172', 'n01768244', 'n01770393', 'n01774384', 'n01774750', 'n01784675', 'n01855672', 'n01882714', 'n01910747', 'n01917289', 'n01944390', 'n01945685', 'n01950731', 'n01983481', 'n01984695', 'n02002724', 'n02056570', 'n02058221', 'n02074367', 'n02085620', 'n02094433', 'n02099601', 'n02099712', 'n02106662', 'n02113799', 'n02123045', 'n02123394', 'n02124075', 'n02125311', 'n02129165', 'n02132136', 'n02165456', 'n02190166', 'n02206856', 'n02226429', 'n02231487', 'n02233338', 'n02236044', 'n02268443', 'n02279972', 'n02281406', 'n02321529', 'n02364673', 'n02395406', 'n02403003', 'n02410509', 'n02415577', 'n02423022', 'n02437312', 'n02480495', 'n02481823', 'n02486410', 'n02504458', 'n02509815', 'n02666196', 'n02669723', 'n02699494', 'n02730930', 'n02769748', 'n02788148', 'n02791270', 'n02793495', 'n02795169', 'n02802426', 'n02808440', 'n02814533', 'n02814860', 'n02815

In [ ]:
## to save the best model 

checkpoint = ModelCheckpoint('/content/gdrive/My Drive/App/Tiny/SGD/imgaug/v3/tiny_class_weights_best_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True)

##callback_list = [checkpoint, LearningRateScheduler(poly_decay)]

callback_list = [checkpoint, clr]

##### Epochs 1 -10

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size,
    callbacks = callback_list,
    #initial_epoch=50,
    class_weight = class_weights
)

Epoch 1/10
390/390 [==============================] - 352s 902ms/step - loss: 2.7367 - acc: 0.7247 - val_loss: 2.6657 - val_acc: 0.4837

Epoch 00001: val_acc improved from -inf to 0.48367, saving model to /content/gdrive/My Drive/App/Tiny/SGD/imgaug/v3/tiny_class_weights_best_model.hdf5
Epoch 2/10
390/390 [==============================] - 339s 868ms/step - loss: 3.2026 - acc: 0.6697 - val_loss: 2.9777 - val_acc: 0.4369

Epoch 00002: val_acc did not improve from 0.48367
Epoch 3/10
390/390 [==============================] - 337s 864ms/step - loss: 3.8574 - acc: 0.6054 - val_loss: 3.0988 - val_acc: 0.4120

Epoch 00003: val_acc did not improve from 0.48367
Epoch 4/10
390/390 [==============================] - 337s 864ms/step - loss: 4.4355 - acc: 0.5525 - val_loss: 2.7673 - val_acc: 0.3914

Epoch 00004: val_acc did not improve from 0.48367
Epoch 5/10
390/390 [==============================] - 337s 864ms/step - loss: 4.8898 - acc: 0.5160 - val_loss: 3.1629 - val_acc: 0.3671

Epoch 00005: v

##### Epochs 11-20

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size,
    callbacks = callback_list,
    initial_epoch=10,
    class_weight = class_weights
)

Epoch 11/20
390/390 [==============================] - 340s 872ms/step - loss: 2.4578 - acc: 0.7422 - val_loss: 2.6218 - val_acc: 0.5069

Epoch 00011: val_acc did not improve from 0.51468
Epoch 12/20
390/390 [==============================] - 337s 865ms/step - loss: 2.7643 - acc: 0.7103 - val_loss: 2.6740 - val_acc: 0.4741

Epoch 00012: val_acc did not improve from 0.51468
Epoch 13/20
390/390 [==============================] - 337s 865ms/step - loss: 3.2474 - acc: 0.6627 - val_loss: 2.7756 - val_acc: 0.4495

Epoch 00013: val_acc did not improve from 0.51468
Epoch 14/20
390/390 [==============================] - 339s 870ms/step - loss: 3.7706 - acc: 0.6122 - val_loss: 2.8600 - val_acc: 0.4227

Epoch 00014: val_acc did not improve from 0.51468
Epoch 15/20
390/390 [==============================] - 338s 868ms/step - loss: 4.2738 - acc: 0.5672 - val_loss: 3.2305 - val_acc: 0.3631

Epoch 00015: val_acc did not improve from 0.51468
Epoch 16/20
390/390 [==============================] - 337s 

##### Epochs 21-35

In [ ]:
##### CLR #########
## Experiment with CLR instead of poly decay
clr = CyclicLR(base_lr=0.001, max_lr=0.1, step_size=4000, mode='exp_range')

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=35,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size,
    callbacks = callback_list,
    initial_epoch=20,
    class_weight = class_weights
)

Epoch 21/35
390/390 [==============================] - 345s 885ms/step - loss: 2.2651 - acc: 0.7608 - val_loss: 2.5768 - val_acc: 0.5229

Epoch 00021: val_acc improved from 0.51468 to 0.52289, saving model to /content/gdrive/My Drive/App/Tiny/SGD/imgaug/v3/tiny_class_weights_best_model.hdf5
Epoch 22/35
390/390 [==============================] - 344s 881ms/step - loss: 2.4187 - acc: 0.7451 - val_loss: 2.7462 - val_acc: 0.4939

Epoch 00022: val_acc did not improve from 0.52289
Epoch 23/35
390/390 [==============================] - 346s 886ms/step - loss: 2.8189 - acc: 0.7039 - val_loss: 2.9392 - val_acc: 0.4552

Epoch 00023: val_acc did not improve from 0.52289
Epoch 24/35
390/390 [==============================] - 345s 884ms/step - loss: 3.3042 - acc: 0.6552 - val_loss: 3.0406 - val_acc: 0.4193

Epoch 00024: val_acc did not improve from 0.52289
Epoch 25/35
390/390 [==============================] - 344s 882ms/step - loss: 3.8268 - acc: 0.6056 - val_loss: 3.2471 - val_acc: 0.3750

Epoch 

## Score

In [ ]:
#!cp '/content/gdrive/My Drive/App/Tiny/SGD/imgaug/imgaug_clr_e136.hdf5' /content

In [ ]:
#!cp '/content/gdrive/My Drive/App/Tiny/SGD/imgaug/tiny_custom_best_model.hdf5' /content

In [ ]:
!cp '/content/gdrive/My Drive/App/Tiny/SGD/imgaug/v3/tiny_class_weights_best_model.hdf5' /content

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('/content/tiny_class_weights_best_model.hdf5')

In [ ]:
validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)


score=model.evaluate_generator(generator=validation_generator, steps = num_validation_samples)
print(score)

Found 10000 images belonging to 200 classes.
[2.587500740671457, 0.5224]


## Summary

* Started with ResNet50. The Accuracy was around 40%
* For this dataset, there is a **need to shorter the deepth of the network**.
  Used Custom ResNet by **ignoring the last set of repetition blocks.**
* Explored mulitple versions for ResNet and attempted to make the network as much **wide **as possible by avoiding the Out of Memory error from Colab.
* Also, used SeparableConv2D(), which makes it less number of parameters.
* Explored optimizer Adam and SGD.
* Explored Cyclic LR and Poly decay.
* The **Cyclic LR gave a boost of 2% **when compared with Poly decay with SGD.
* Integrated library **imgaug for image augmentation**.
* The final validation accuracy obtained is **52.24%**


## Future Direction

* Identify a **better and easy way to analyze** the results. Idea is to cut down the analysis time.
  (HTML reporting). **Need to build a package to aid analysis if not available already?**
* Given an image /class of images, how to find out the otuput of the different Conv layers.
  Would that help in improving the accuray? Any **debugger for Computer Vision** out there? 
* Find out a mechanism for **LR finder**. With less experiment, identify the range for LR.
* Identify the ways to improve the accuracy for this dataset.